In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import glob
import warnings
import anndata as ad
warnings.filterwarnings("ignore")

In [ ]:
h5_files = glob.glob("/exports/archive/hg-funcgenom-research/Genelab_dataset/"
                    + "OSD-562/*_output/"
                     + "outs/raw_feature_bc_matrix/")

In [ ]:
adatas = []
for file in h5_files:
    tmp_adata = sc.read_10x_mtx(file)
    tmp_adata.var_names_make_unique()
    tmp_adata.obs_names_make_unique()
    tmp_adata.obs['sample'] = file.split('/')[6].rsplit('_', 1)[0]
    sc.pp.filter_cells(adata, min_genes=100)
    sc.pp.filter_genes(tmp_adata, min_cells=3)
    adatas.append(tmp_adata)
adata = ad.concat(adatas, axis=0)

In [ ]:
adata.obs['sample'].unique()

In [ ]:
sample_information = pd.read_csv("/exports/archive/hg-funcgenom-research" + 
                                 "/Genelab_dataset/OSD-562/OSD-562-samples.csv")

In [ ]:
sample_information['Sample Name'] = sample_information['Sample Name'].str.split("RRRM2_HPC_", expand=True)[1]
sample_information.set_index('Sample Name', inplace=True)

The downloaded data contains one extra sample not present in the Metadata dataframe, we proceed only with the overlap

In [ ]:
overlap = np.intersect1d(adata.obs['sample'].unique(), sample_information.index)
overlap

In [ ]:
new_adatas = []
for sample in overlap:
    adata_subset = adata[adata.obs['sample'] == sample].copy()    
    sample_metadata = sample_information.loc[sample]
    for col in sample_metadata.index:
        adata_subset.obs[col] = sample_metadata[col]

    new_adatas.append(adata_subset)

In [ ]:
adata = ad.concat(new_adatas, axis=0)

In [ ]:
adata.var["mt"] = adata.var_names.str.startswith("mt-")
adata.var["ribo"] = adata.var_names.str.startswith(("Rps", "Rpl"))
adata.var["hb"] = adata.var_names.str.contains("^Hb[^(p)]")

In [ ]:
adata

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

In [ ]:
adata

In [ ]:
adata.write_h5ad("/exports/archive/hg-funcgenom-research/Genelab_dataset/OSD-562/adata_raw.h5ad")